# ***YOLOv12 + EfficientNetB0 + PANet***

# YOLOv12n EfficientNet-B0 + PANet
**Varian paling akurat di kelas < 6M params!**

Keunggulan EfficientNet-B0 + PANet:
- EfficientNet-B0 pretrained → backbone paling efisien & akurat di ukurannya
- Full PANet + A2C2f → bidirectional fusion terbaik untuk dataset medis kompleks
- Recall sangat tinggi → minim missed caries (kritis untuk screening)
- mAP50-95 biasanya tembus 0.68–0.71+ (v10 dataset) → masuk top-tier
- Inference masih cepat: ~18–22 ms di RTX 4090, ~45 ms di CPU modern

## 1. Install Library Terbaru

In [2]:
!pip install -q ultralytics optuna pandas roboflow PyYAML torch torchvision --upgrade

import optuna
from ultralytics import YOLO
from pathlib import Path
from roboflow import Roboflow
import yaml
import json
import pandas as pd

## 2. Pengaturan Global

In [3]:
VARIASI_NAME     = "yolo12s_efficientnetb0_panet"
VERSION_NUM      = 13
EPOCHS_SEARCH    = 20
EPOCHS_FINAL     = 500
BATCH_CANDIDATES = [16, 24, 32, 48, 64]

OPTUNA_ROOT      = Path("runs/optuna")
VARIASI_DIR      = OPTUNA_ROOT / VARIASI_NAME
VARIASI_DIR.mkdir(parents=True, exist_ok=True)

DB_PATH          = VARIASI_DIR / "optuna_study.db"
JSON_PATH        = VARIASI_DIR / "best_hyperparameters.json"
CSV_PATH         = VARIASI_DIR / "optuna_trials_log.csv"
YAML_PATH        = VARIASI_DIR / "yolo12s_efficientnetb0_panet.yaml"

## 3. Custom YAML: EfficientNet-B0 Backbone + Full PANet

In [4]:
yaml_content = """
nc: 7
scales:
  n: [0.50, 0.25, 1024]
  s: [0.50, 0.50, 1024]
  m: [0.50, 1.00, 512]
  l: [1.00, 1.00, 512]
  x: [1.00, 1.50, 512]

backbone:
  - [-1, 1, TorchVision, [0, efficientnet_b0, DEFAULT, True, 2, True]]  # 0: EfficientNet-B0
  - [0, 1, Index, [40, 4]]           # 1: P3 raw (40ch, /8)
  - [1, 1, Conv, [256, 1, 1]]        # 2: P3 → 256ch
  - [0, 1, Index, [112, 6]]          # 3: P4 raw (112ch, /16)
  - [3, 1, Conv, [512, 1, 1]]        # 4: P4 → 512ch
  - [0, 1, Index, [320, 8]]          # 5: P5 raw (320ch, /32)
  - [5, 1, Conv, [1024, 1, 1]]       # 6: P5 → 1024ch
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]      # 7
  - [[-1, 4], 1, Concat, [1]]                        # 8 cat P5_up + P4
  - [-1, 2, A2C2f, [512, False, -1]]                 # 9 refined P4
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]      # 10
  - [[-1, 2], 1, Concat, [1]]                        # 11 cat P4_up + P3
  - [-1, 2, A2C2f, [256, False, -1]]                 # 12 refined P3
  - [-1, 1, Conv, [256, 3, 2]]                       # 13 down
  - [[-1, 9], 1, Concat, [1]]                        # 14
  - [-1, 2, A2C2f, [512, False, -1]]                 # 15 refined P4
  - [-1, 1, Conv, [512, 3, 2]]                       # 16 down
  - [[-1, 6], 1, Concat, [1]]                        # 17
  - [-1, 2, C3k2, [1024, True]]                      # 18 refined P5
  - [[12, 15, 18], 1, Detect, [nc]]                  # Detect P3,P4,P5
"""
with open(YAML_PATH, "w") as f:
    f.write(yaml_content.strip() + "\n")
print(f"EfficientNet-B0 + PANet YAML berhasil dibuat → {YAML_PATH}")

EfficientNet-B0 + PANet YAML berhasil dibuat → runs/optuna/yolo12s_efficientnetb0_panet/yolo12s_efficientnetb0_panet.yaml


## 4. Download Dataset Dental Caries

In [5]:
API_KEY   = "QOd5ldAdjiaehHn5m6WC"
WORKSPACE = "dentalogic8"
PROJECT_ID = "dental-caries-7kttb"

rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT_ID)
dataset = project.version(VERSION_NUM).download("yolov12")

DATASET_DIR = Path(f"dental-caries-{VERSION_NUM}")
DATA_YAML   = DATASET_DIR / "data.yaml"

if not DATA_YAML.exists():
    raise FileNotFoundError("data.yaml tidak ditemukan!")

print(f"Dataset v{VERSION_NUM} siap → {DATA_YAML}")

loading Roboflow workspace...
loading Roboflow project...
Dataset v13 siap → dental-caries-13/data.yaml


## 5. Inisialisasi CSV + Combined Score
Bobot:
- **50%** mAP50-95 → akurasi keseluruhan
- **30%** Recall → kritis (tidak boleh ada karies terlewat)
- **20%** Precision → kontrol false positive

In [6]:
def init_csv():
    if not CSV_PATH.exists():
        pd.DataFrame(columns=[
            "Trial","Batch","Epochs_Completed","lr0","lrf","weight_decay",
            "box","cls","dfl","mAP@50","mAP@50-95","Precision","Recall",
            "Combined_Score","Status"
        ]).to_csv(CSV_PATH, index=False)
        print(f"CSV log dibuat → {CSV_PATH}")
init_csv()

def get_score(m):
    map5095 = m.get("metrics/mAP50-95(B)", 0.0)
    recall  = m.get("metrics/recall(B)", 0.0)
    prec    = m.get("metrics/precision(B)", 0.0)
    return 0.5*map5095 + 0.3*recall + 0.2*prec, map5095, recall, prec

CSV log dibuat → runs/optuna/yolo12s_efficientnetb0_panet/optuna_trials_log.csv


## 6. Optuna Hyperparameter Search – EfficientNet-B0 + PANet

In [ ]:
study = optuna.create_study(
    study_name=f"dental_caries_{VARIASI_NAME}",
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=42),
    storage=f"sqlite:///{DB_PATH}",
    load_if_exists=True
)

def objective(trial):
    batch = trial.suggest_categorical("batch", BATCH_CANDIDATES)
    lr0   = trial.suggest_float("lr0", 1e-4, 0.1, log=True)
    lrf   = trial.suggest_float("lrf", 0.005, 0.2, log=True)
    wd    = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)
    box   = trial.suggest_float("box", 7.5, 25.0)
    cls   = trial.suggest_float("cls", 0.3, 1.5)
    dfl   = trial.suggest_float("dfl", 0.5, 4.0)

    model = YOLO(str(YAML_PATH))
    try:
        results = model.train(
            data=str(DATA_YAML),
            imgsz=640,
            epochs=EPOCHS_SEARCH,
            batch=batch,
            device=3,
            project=str(VARIASI_DIR),
            name=f"trial_{trial.number:04d}",
            exist_ok=True,
            pretrained=False,
            optimizer="AdamW",
            lr0=lr0, lrf=lrf, weight_decay=wd,
            box=box, cls=cls, dfl=dfl,
            patience=10,
            deterministic=True,
            cache="disk",
            verbose=False,
            plots=False,
            close_mosaic=5,
            amp=True
        )
        m = results.results_dict
        score, map5095, recall, prec = get_score(m)
        map50 = m.get("metrics/mAP50(B)", 0.0)
        status = "Completed"
    except Exception as e:
        print(f"Trial {trial.number} GAGAL → {str(e)[:100]}")
        score = map5095 = recall = prec = map50 = 0.0
        status = "Failed"

    pd.DataFrame([{
        "Trial": trial.number, "Batch": batch, "Epochs_Completed": EPOCHS_SEARCH,
        "lr0": round(lr0,8), "lrf": round(lrf,6), "weight_decay": wd,
        "box": round(box,4), "cls": round(cls,4), "dfl": round(dfl,4),
        "mAP@50": round(map50,5), "mAP@50-95": round(map5095,5),
        "Precision": round(prec,5), "Recall": round(recall,5),
        "Combined_Score": round(score,6), "Status": status
    }]).to_csv(CSV_PATH, mode='a', header=False, index=False)

    print(f"Trial {trial.number:3d} │ B{batch:3d} │ mAP50-95 {map5095:.5f} │ Score {score:.6f}")
    return score

print(f"\n{'='*95}")
print(f"OPTUNA SEARCH: {VARIASI_NAME.upper()} (EFFICIENTNET-B0 + PANET)")
print(f"YAML : {YAML_PATH.name}")
print(f"Trial epochs: {EPOCHS_SEARCH} │ Final epochs: {EPOCHS_FINAL}")
print(f"{'='*95}\n")

study.optimize(objective, n_trials=20)

[I 2025-11-22 18:13:12,251] A new study created in RDB with name: dental_caries_yolo12s_efficientnetb0_panet



OPTUNA SEARCH: YOLO12S_EFFICIENTNETB0_PANET (EFFICIENTNET-B0 + PANET)
YAML : yolo12s_efficientnetb0_panet.yaml
Trial epochs: 20 │ Final epochs: 500

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /raid/tegar/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20.5M/20.5M [00:00<00:00, 28.7MB/s]


Ultralytics 8.3.230 🚀 Python-3.10.12 torch-2.9.1+cu128 CUDA:3 (NVIDIA H200, 143167MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=18.019512705506152, cache=disk, cfg=None, classes=None, close_mosaic=5, cls=1.1496870933552545, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dental-caries-13/data.yaml, degrees=0.0, deterministic=True, device=3, dfl=0.5720457300353086, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00029375384576328325, lrf=0.006194745024628934, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/optuna/yolo12s_efficientnetb0_panet/yolo12s_efficientnetb0_panet.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name

## 7. Simpan Best Hyperparameter

In [ ]:
with open(JSON_PATH, "w") as f:
    json.dump(study.best_params, f, indent=4)
print(f"\nOPTIMASI SELESAI! Best Score: {study.best_value:.6f}")
print(f"Best params → {JSON_PATH}")
print(f"Log lengkap → {CSV_PATH}")

## 8. Final Training → EfficientNet-B0 + PANet

In [ ]:
!pip install -q ultralytics --upgrade
from ultralytics import YOLO
import json
from pathlib import Path

with open(JSON_PATH) as f:
    best = json.load(f)

print("Best Hyperparameters (EfficientNet-B0 + PANet):")
for k, v in best.items():
    print(f" {k:12} → {v}")

model = YOLO(str(YAML_PATH))
model.train(
    data="dental-caries-10/data.yaml",
    imgsz=640,
    epochs=EPOCHS_FINAL,
    batch=best["batch"],
    device=0,
    project="runs/final",
    name=f"{VARIASI_NAME}_FINAL",
    exist_ok=True,
    pretrained=False,
    optimizer="AdamW",
    patience=50,
    cache="disk",
    plots=True,
    save=True,
    close_mosaic=50,
    lr0=best["lr0"],
    lrf=best["lrf"],
    weight_decay=best["weight_decay"],
    box=best["box"],
    cls=best["cls"],
    dfl=best["dfl"],
)
print(f"\nFINAL TRAINING EfficientNet-B0 + PANet SELESAI!")
print(f"Model terbaik → runs/final/{VARIASI_NAME}_FINAL/weights/best.pt")